## Step1: initialization, import necessary packages
- files are mounted from local file system "volgenmodel-docker/data" into the docker container path "/home/data" (see "docker-compose.yml")

In [1]:
%pip install tqdm
import os
import sys
sys.path.append('/home/data')
from os.path import join
from functions import *
import tifffile
import SimpleITK as sitk
import numpy as np
import tifffile
import nibabel as nib
import numpy as np
from tqdm.auto import tqdm

os.getcwd()
os.chdir("/volgenmodel-nipype")
print(f"I'm working in folder: {os.getcwd()}")

## folder initialization
input_folder   = '/home/data/masked_brains'
output_folder  = '/home/data/output_templates'
process_folder = '/home/data/converted_brains'
tmp_folder     = '/home/data/tmp'
result_folder  = '/volgenmodel-nipype'

## parameter init
parameters={'fit_stages': 'lin,1,2', ## for testing 'lin,1,2' for best quality use larger numbers like 'lin,1,1,2,2,3,3,4,5,6' from https://doi.org/10.1016/j.ymeth.2015.01.005
            'ncpus': 8, ## how many cores should the template creation use? be careful, don't use to many or the system could crash
            'img_thresholds': len(fileList(input_folder))*[1], ## the threshold value should be the smallest grey value that is located at one of the edges of the image. 
            'padding_size': 25, ## Padding-Size should be ~10% of the largest dimension (88x150x249 -> paddingSize = 25)
            'min_number_of_brains_in_template': 9
                  }

Note: you may need to restart the kernel to use updated packages.
I'm working in folder: /volgenmodel-nipype


## Step2: N4 (identical to N3) bias field correction 
- https://simpleitk.readthedocs.io/en/master/link_N4BiasFieldCorrection_docs.html


In [2]:
files = fileList(input_folder)
files = [join(input_folder,file) for file in files 
         if file.find("_N4") == -1]
out_path = input_folder

for _, mri_file in tqdm(enumerate(files)):
    outputImagePath = join(input_folder, mri_file.split("/")[-1].split(".")[0] + '_N4.nii')
    print(outputImagePath)
    create_N4_img(mri_file, outputImagePath)

0it [00:00, ?it/s]

/home/data/masked_brains/211108_SK_N4.nii
N4 bias correction runs.
Finished N4 Bias Field Correction to /home/data/masked_brains/211108_SK_N4.nii
/home/data/masked_brains/211111_SK_N4.nii
N4 bias correction runs.
Finished N4 Bias Field Correction to /home/data/masked_brains/211111_SK_N4.nii
/home/data/masked_brains/211112_SK_N4.nii
N4 bias correction runs.
Finished N4 Bias Field Correction to /home/data/masked_brains/211112_SK_N4.nii
/home/data/masked_brains/211115_noArtefact_SK_N4.nii
N4 bias correction runs.
Finished N4 Bias Field Correction to /home/data/masked_brains/211115_noArtefact_SK_N4.nii
/home/data/masked_brains/211128_SK_N4.nii
N4 bias correction runs.
Finished N4 Bias Field Correction to /home/data/masked_brains/211128_SK_N4.nii
/home/data/masked_brains/211129_noArtefact_SK_N4.nii
N4 bias correction runs.
Finished N4 Bias Field Correction to /home/data/masked_brains/211129_noArtefact_SK_N4.nii
/home/data/masked_brains/THG6413_SK_N4.nii
N4 bias correction runs.
Finished N4 

## Step 3: cropping based on defined threshold or auto-cropping (if available)
- to find a proper threshold load the image into imageJ and find the lowest grey-value on the border of the tissue (the croping will be done in a cubic shape from the outside)
- you can try to let the function find the threshold by themself by setting: "thresholds='auto'" (TODO: test the auto threshold function!)

In [2]:
files = fileList(input_folder)
files = [join(input_folder,file) for file in files 
         if file.endswith("nii") and file.find("_N4.") != -1]
out_path = input_folder

for i, mri_file in tqdm(enumerate(files)):    
    outputImagePath = join(input_folder, mri_file.split("/")[-1].split(".")[0] + '_auto-crop.nii')
    """
    create_cropped_imgs(input_folder, 
                       input_folder, 
                       thresholds='auto', 
                       save_as_dtype=np.int16)
    """
    create_cropped_imgs(mri_file, 
                       outputImagePath, 
                       threshold=parameters['img_thresholds'][i], 
                       save_as_dtype=np.int32)

0it [00:00, ?it/s]

Processing image: /home/data/masked_brains/211108_SK_N4.nii
Initial Dims: (250, 250, 120)
0.3024328771952782
current threshold is 1
Initial bestBorders: [124, 126, 59, 61]
Final bestBorders: [0, 239, 20, 107]
Number of slices after: 154
Final Dims: (154, 240, 88)
I saved to /home/data/masked_brains/211108_SK_N4_auto-crop.nii
Processing image: /home/data/masked_brains/211111_SK_N4.nii
Initial Dims: (250, 250, 120)
0.36275176362064404
current threshold is 1
Initial bestBorders: [124, 126, 59, 61]
Final bestBorders: [0, 234, 15, 104]
Number of slices after: 154
Final Dims: (154, 235, 90)
I saved to /home/data/masked_brains/211111_SK_N4_auto-crop.nii
Processing image: /home/data/masked_brains/211112_SK_N4.nii
Initial Dims: (250, 250, 120)
0.4026731875215986
current threshold is 1
Initial bestBorders: [124, 126, 59, 61]
Final bestBorders: [0, 228, 15, 101]
Number of slices after: 150
Final Dims: (150, 229, 87)
I saved to /home/data/masked_brains/211112_SK_N4_auto-crop.nii
Processing image: 

## Step 4: Zero-Padding
- Padding-Size should be ~10% of the largest dimension (88x150x249 -> paddingSize = 25)

In [3]:
files = fileList(input_folder)
files = [join(input_folder,file) for file in files 
         if file.endswith("nii") and file.find("_N4_auto-crop.") != -1]
out_path = input_folder

for _, mri_file in tqdm(enumerate(files)):
    outputImagePath = join(input_folder, mri_file.split("/")[-1].split(".")[0] + "_padded.nii")
    print(outputImagePath)
    create_padding_imgs(mri_file, outputImagePath, init_paddingSize='10%')


0it [00:00, ?it/s]

/home/data/masked_brains/211108_SK_N4_auto-crop_padded.nii
/home/data/masked_brains/211108_SK_N4_auto-crop.nii
I padded file: /home/data/masked_brains/211108_SK_N4_auto-crop_padded.nii size of 25 in all 3D!
/home/data/masked_brains/211111_SK_N4_auto-crop_padded.nii
/home/data/masked_brains/211111_SK_N4_auto-crop.nii
I padded file: /home/data/masked_brains/211111_SK_N4_auto-crop_padded.nii size of 24 in all 3D!
/home/data/masked_brains/211112_SK_N4_auto-crop_padded.nii
/home/data/masked_brains/211112_SK_N4_auto-crop.nii
I padded file: /home/data/masked_brains/211112_SK_N4_auto-crop_padded.nii size of 23 in all 3D!
/home/data/masked_brains/211115_noArtefact_SK_N4_auto-crop_padded.nii
/home/data/masked_brains/211115_noArtefact_SK_N4_auto-crop.nii
I padded file: /home/data/masked_brains/211115_noArtefact_SK_N4_auto-crop_padded.nii size of 24 in all 3D!
/home/data/masked_brains/211128_SK_N4_auto-crop_padded.nii
/home/data/masked_brains/211128_SK_N4_auto-crop.nii
I padded file: /home/data/ma

## Step 5: Nifti to Minc convert
- it also delete possible unused mnc files in the processed folder to prevent issues

In [2]:
nii_to_minc(input_folder, process_folder)
delete_files(process_folder, 'mnc') ## delete unused .nii files in processed folder

nii2mnc -unsigned -float -clobber /home/data/converted_brains/mouse0 /home/data/converted_brains/mouse0.mnc
Using s-form transform:
  1.0000,   0.0000,   0.0000,   0.0000, 
  0.0000,   1.0000,   0.0000,   0.0000, 
  0.0000,   0.0000,   1.0000,   0.0000, 
  0.0000,   0.0000,   0.0000,   1.0000, 

name   | start      | step    | cosines
zspace |     0.0000 |  1.0000 |  0.0000  0.0000  1.0000
yspace |     0.0000 |  1.0000 |  0.0000  1.0000  0.0000
xspace |     0.0000 |  1.0000 |  1.0000  0.0000  0.0000
Performing voxel format conversion.
nii2mnc -unsigned -float -clobber /home/data/converted_brains/mouse1 /home/data/converted_brains/mouse1.mnc


<nifti_image
  nifti_type = 'NIFTI-1+'
  header_filename = '/home/data/converted_brains/mouse0.nii'
  image_filename = '/home/data/converted_brains/mouse0.nii'
  image_offset = '352'
  ndim = '3'
  nx = '204'
  ny = '290'
  nz = '138'
  dx = '1'
  dy = '1'
  dz = '1'
  datatype = '16'
  datatype_name = 'FLOAT32'
  nvox = '8164080'
  nbyper = '4'
  byteorder = 'LSB_FIRST'
  scl_slope = '1'
  scl_inter = '0'
  sform_code = '1'
  sform_code_name = 'Scanner Anat'
  sto_xyz_matrix = '1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1'
  sto_ijk matrix = '1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1'
  sform_i_orientation = 'Left-to-Right'
  sform_j_orientation = 'Posterior-to-Anterior'
  sform_k_orientation = 'Inferior-to-Superior'
  num_ext = '0'
/>
<nifti_image
  nifti_type = 'NIFTI-1+'
  header_filename = '/home/data/converted_brains/mouse1.nii'
  image_filename = '/home/data/converted_brains/mouse1.nii'
  image_offset = '352'
  ndim = '3'
  nx = '202'
  ny = '283'
  nz = '138'
  dx = '1'
  dy = '1'
  dz = '1'
  dataty

Using s-form transform:
  1.0000,   0.0000,   0.0000,   0.0000, 
  0.0000,   1.0000,   0.0000,   0.0000, 
  0.0000,   0.0000,   1.0000,   0.0000, 
  0.0000,   0.0000,   0.0000,   1.0000, 

name   | start      | step    | cosines
zspace |     0.0000 |  1.0000 |  0.0000  0.0000  1.0000
yspace |     0.0000 |  1.0000 |  0.0000  1.0000  0.0000
xspace |     0.0000 |  1.0000 |  1.0000  0.0000  0.0000
Performing voxel format conversion.
nii2mnc -unsigned -float -clobber /home/data/converted_brains/mouse2 /home/data/converted_brains/mouse2.mnc
Using s-form transform:
  1.0000,   0.0000,   0.0000,   0.0000, 
  0.0000,   1.0000,   0.0000,   0.0000, 
  0.0000,   0.0000,   1.0000,   0.0000, 
  0.0000,   0.0000,   0.0000,   1.0000, 

name   | start      | step    | cosines
zspace |     0.0000 |  1.0000 |  0.0000  0.0000  1.0000
yspace |     0.0000 |  1.0000 |  0.0000  1.0000  0.0000
xspace |     0.0000 |  1.0000 |  1.0000  0.0000  0.0000
Performing voxel format conversion.
nii2mnc -unsigned -float -

<nifti_image
  nifti_type = 'NIFTI-1+'
  header_filename = '/home/data/converted_brains/mouse2.nii'
  image_filename = '/home/data/converted_brains/mouse2.nii'
  image_offset = '352'
  ndim = '3'
  nx = '196'
  ny = '275'
  nz = '133'
  dx = '1'
  dy = '1'
  dz = '1'
  datatype = '16'
  datatype_name = 'FLOAT32'
  nvox = '7168700'
  nbyper = '4'
  byteorder = 'LSB_FIRST'
  scl_slope = '1'
  scl_inter = '0'
  sform_code = '1'
  sform_code_name = 'Scanner Anat'
  sto_xyz_matrix = '1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1'
  sto_ijk matrix = '1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1'
  sform_i_orientation = 'Left-to-Right'
  sform_j_orientation = 'Posterior-to-Anterior'
  sform_k_orientation = 'Inferior-to-Superior'
  num_ext = '0'
/>
<nifti_image
  nifti_type = 'NIFTI-1+'
  header_filename = '/home/data/converted_brains/mouse3.nii'
  image_filename = '/home/data/converted_brains/mouse3.nii'
  image_offset = '352'
  ndim = '3'
  nx = '201'
  ny = '278'
  nz = '138'
  dx = '1'
  dy = '1'
  dz = '1'
  dataty

Using s-form transform:
  1.0000,   0.0000,   0.0000,   0.0000, 
  0.0000,   1.0000,   0.0000,   0.0000, 
  0.0000,   0.0000,   1.0000,   0.0000, 
  0.0000,   0.0000,   0.0000,   1.0000, 

name   | start      | step    | cosines
zspace |     0.0000 |  1.0000 |  0.0000  0.0000  1.0000
yspace |     0.0000 |  1.0000 |  0.0000  1.0000  0.0000
xspace |     0.0000 |  1.0000 |  1.0000  0.0000  0.0000
Performing voxel format conversion.
nii2mnc -unsigned -float -clobber /home/data/converted_brains/mouse4 /home/data/converted_brains/mouse4.mnc
Using s-form transform:
  1.0000,   0.0000,   0.0000,   0.0000, 
  0.0000,   1.0000,   0.0000,   0.0000, 
  0.0000,   0.0000,   1.0000,   0.0000, 
  0.0000,   0.0000,   0.0000,   1.0000, 

name   | start      | step    | cosines
zspace |     0.0000 |  1.0000 |  0.0000  0.0000  1.0000
yspace |     0.0000 |  1.0000 |  0.0000  1.0000  0.0000
xspace |     0.0000 |  1.0000 |  1.0000  0.0000  0.0000
Performing voxel format conversion.
nii2mnc -unsigned -float -

...

Copying chunks:..........................................................................................................................................Done.
Copying chunks:.

mouse7.mnc -> reworked_mouse_mouse7.mnc


.........................................................................................................................................

mouse8.mnc -> reworked_mouse_mouse8.mnc


Done.


## Step 6: Create the Templates
- set up the create_parameters if necessary
- use only a small version of the fit_stages (lin,1,2) to prevent exessive calculation time
- for the best version of the template you should use the parameterset: "lin,1,2,3,4,5,6,7,8"

In [2]:
parameters

{'fit_stages': 'lin,1,2',
 'ncpus': 8,
 'img_thresholds': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'padding_size': 25,
 'min_number_of_brains_in_template': 9}

In [2]:
delete_files(tmp_folder, 'mnc') ## delete unused .nii files in processed folder
delete_files(tmp_folder, 'nii') ## delete unused .nii files in processed folder
process_files(process_folder, output_folder, tmp_folder, parameters, result_folder)

['/home/data/converted_brains/mouse0.mnc', '/home/data/converted_brains/mouse1.mnc', '/home/data/converted_brains/mouse2.mnc', '/home/data/converted_brains/mouse3.mnc', '/home/data/converted_brains/mouse4.mnc', '/home/data/converted_brains/mouse5.mnc', '/home/data/converted_brains/mouse6.mnc', '/home/data/converted_brains/mouse7.mnc', '/home/data/converted_brains/mouse8.mnc']
cp /home/data/converted_brains/mouse0.mnc /home/data/tmp
cp /home/data/converted_brains/mouse1.mnc /home/data/tmp
cp /home/data/converted_brains/mouse2.mnc /home/data/tmp
cp /home/data/converted_brains/mouse3.mnc /home/data/tmp
cp /home/data/converted_brains/mouse4.mnc /home/data/tmp
cp /home/data/converted_brains/mouse5.mnc /home/data/tmp
cp /home/data/converted_brains/mouse6.mnc /home/data/tmp
cp /home/data/converted_brains/mouse7.mnc /home/data/tmp
cp /home/data/converted_brains/mouse8.mnc /home/data/tmp
250630-11:11:11,191 nipype.workflow INFO:
	 [Node] Setting-up "datasource" in "/tmp/tmp1o6wpwr1/datasource".

RuntimeError: Traceback (most recent call last):
  File "/opt/conda/envs/myenv/lib/python3.8/site-packages/nipype/pipeline/plugins/multiproc.py", line 67, in run_node
    result["result"] = node.run(updatehash=updatehash)
  File "/opt/conda/envs/myenv/lib/python3.8/site-packages/nipype/pipeline/engine/nodes.py", line 527, in run
    result = self._run_interface(execute=True)
  File "/opt/conda/envs/myenv/lib/python3.8/site-packages/nipype/pipeline/engine/nodes.py", line 645, in _run_interface
    return self._run_command(execute)
  File "/opt/conda/envs/myenv/lib/python3.8/site-packages/nipype/pipeline/engine/nodes.py", line 771, in _run_command
    raise NodeExecutionError(msg)
nipype.pipeline.engine.nodes.NodeExecutionError: Exception raised while executing Node _register_00_0.

Cmdline:
	bestlinreg -clobber /volgenmodel-nipype/workflow_temp_workflowMultiProc8/voliso_00_/mouse0_volcentre_norm_volpad_bluroutput_blur_norm_voliso.mnc /volgenmodel-nipype/workflow_temp_workflowMultiProc8/

The last modified subfolder is: /volgenmodel-nipype/workflow_temp_workflowMultiProc8
Last modification time: 2025-06-30 11:12:20
No valid output folder found: /volgenmodel-nipype/workflow_output_workflowMultiProc8
0


0

## Step X: Remove the workflow temp or output files to restart the calculation from scratch or to save space
- will not delete the files copied to the volume outside the docker container!

In [91]:
delete_workflow_temp(search_folder="/volgenmodel-nipype/")
os.system('rm /volgenmodel-nipype/workflow_output_workflowMultiProc50/ -rf')

0

## Additional Test, check out the format of the example files from: "volgenmodel-fast-example" 
- from: https://github.com/carlohamalainen/volgenmodel-fast-example
- to load them with Fiji you have to convert em to Nifti

In [ ]:
##artifical test with test_examples (so they have to convert to nii to do the whole pipeline)
"""
testlist = os.listdir(input_folder)
for t in testlist:
    it = join(input_folder,t)
    cmd = " ".join(["mnc2nii",it, join(input_folder,t.replace("mnc","nii"))])
    os.system(cmd)
"""